## Contours and mask on Napari

### Dependencies
* Napari should be installed on extra:

```
pip install napari
```

In [ ]:
%load_ext autoreload
%autoreload 2

import napari
import numpy as np
import pandas as pd
from cryocat import cryomap
from cryocat import surfsamp


### Input data
- The data for this tutorial can be downloaded [here](https://oc.biophys.mpg.de/owncloud/s/XRL6qqNbMQ4FH6e).
- There's no expected output for this tutorial since the drawing is user-generated. However, you can refer to 'inputs/040_1_shape.csv' as an example shape from us for this tomogram and 'inputs/masks/040_generated_mask2.mrc' on owncloud as an example mask.

### Initialize the napari viewer and load tomogram

In [ ]:
#Initialize the napari viewer:
view = napari.Viewer()
# Load tomogram from owncloud and add it to the viewer
tomo = cryomap.read('040.mrc')
view.add_image(tomo)

### Contours in Napari

#### Draw contour on Napari
1. In the Napari GUI, click the small cube button at the bottom left of the screen to change the order of the visible axes. You can also adjust the visualization contrast using the contrast limit.
![step_1](./media/label_step1.png)
2. Create a new shape layer using the polygon button located in the middle left. Then, draw the contour using the "Add Polygons" button.
Drawing contours on multiple axes can improve the accuracy of sampling points on the target surfaces. For a detailed guide on using shape layer, refer to page [here](https://napari.org/dev/howtos/layers/shapes.html)
![step_2](./media/label_step2.png)


#### Save shapes into CSV
Once you've finished drawing, you can save your shapes to a CSV file by running the following code. The CSV file can later be loaded into Napari for modifications and visualization.

In [ ]:
def save_shapes(shape_layer, output_file_name):
    """This function saves the shape layer data into a csv file.

    Parameters
    ----------
    shape_layer : list
        A list of numpy arrays where each array represents a shape layer. Each array is expected to have three columns representing the x, y, and z coordinates respectively.
    output_file_name : str
        The name of the output file where the shape layer data will be saved.

    Returns
    -------
    None

    Notes
    -----
    The function creates a pandas DataFrame from the shape layer data and saves it into a csv file. The DataFrame has four columns: 's_id', 'x', 'y', and 'z'. The 's_id' column represents the shape layer id, 'x', 'y', and 'z' columns represent the coordinates of the shape layer.
    """
    x = []
    y = []
    z = []
    s_id = []

    for i in range(len(shape_layer)):
        x.append(shape_layer[i][:, 2])
        y.append(shape_layer[i][:, 1])
        z.append(shape_layer[i][:, 0])
        s_id.append(np.full(shape_layer[i].shape[0], i))

    x = np.concatenate(x, axis=0)
    y = np.concatenate(y, axis=0)
    z = np.concatenate(z, axis=0)
    s_id = np.concatenate(s_id, axis=0)

    # dictionary of lists
    dict = {"s_id": s_id, "x": x, "y": y, "z": z}

    df = pd.DataFrame(dict)
    # saving the dataframe
    df.to_csv(output_file_name, index=False)


# output file name
output = 'inputs/040_1_shape_fromUser.csv'
# saving shapes for future work
shapes_data=view.layers[1].data
save_shapes(shapes_data, output)

#### Read and view shape CSV on napari

In [ ]:
# opening the shape layer which is provided in inputs folder
polygons = surfsamp.SamplePoints.load_shapes('inputs/040_1_shape.csv')
shapes_layer = view.add_shapes(polygons, shape_type='polygon')

### Masks in Napari

#### Draw binary segmentation on napari
Manually drawing a binary mask can be time-consuming. Depending on your dataset, a more efficient solution may be available. Such as using [MemBrain-Seg](https://github.com/teamtomo/membrain-seg) for membrane segmentation in 3D for cryo-ET.

In [ ]:
# create a empty label
label = np.zeros(tomo.shape, dtype=int)
# add label to the viewer
view.add_labels(label)

1. In the Napari GUI, select the label layer and choose the brush tool at the top. You can draw the mask by left-clicking and dragging your mouse. To draw in 3D, set 'n edit dim' to 3. To label multiple objects on same tomogram, assign a different number to each label before drawing. For more information of how to use label in napari, refer to page [here](https://napari.org/dev/howtos/layers/labels.html#)
![step_1mask](./media/mask_step1.png)

#### Save binary segmentation into mrc
Once you've finished drawing, you can save your labels as an MRC file using the following code. The MRC file can later be opened in Napari for further analysis.

In [ ]:
# output file path
output = 'inputs/040_mask_fromUser.mrc'
# saving label for future work
label = view.layers[1].data
cryomap.write(label, output, data_type='<f4')

#### Simple postprocess of binary segmentation
In surfsamp, `process_mask` can perform simple morphological operations on a given mask. It can also be used for manually drawn masks to refine or modify them as needed. There are four available options: 'closing,' 'opening,' 'dilation,' and 'erosion.'

In [ ]:
# for example to dilate the mask for 2 pixels
dil_label = surfsamp.SamplePoints.process_mask(label,2,'dilation',data_type='int')
view.add_labels(dil_label)

#### Read and view mask as label on napari

In [ ]:
# opening the mask downloaded from owncloud
input = '040_generated_mask_2.mrc'
mask = cryomap.read(input, data_type='int')
view.add_labels(mask)